In [1]:
import numpy as np
import typing

In [2]:
from transformation import *
from similarity import *


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Combal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Combal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Combal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
fun = transformation_functions()
x = "The Boots and the Cats.  "
y = "28/10/2006"
name = lambda x: list(map(lambda x: x.__name__,x))
compose = lambda acc,l : ([acc := x(acc) for x in l],acc)

#lift = lambda f,l : list(map(f,l))
#lift_if_list = lambda f,x : f(x) if isinstance(x,str) else lift(f,x)
#clean = lambda x: [e for e in x if e]

In [4]:
tf = np.random.choice(transformation_functions(),8,replace=False)
_,res = compose("The Boots and the Cats.  ",tf)
print(name(tf))
print(x,res)  

['remove_whitespace', 'remove_stopwords', 'tokenize', 'lowercase', 'remove_punctuation', 'flatten', 'strip_whitespace', 'stem']
The Boots and the Cats.   thebootsandthecat


In [5]:
def flatten(x : valueType) -> str:
    return reduce(lambda acc,x : acc+x,x,"")

In [ ]:
###---------------------------------------------------------------------###
###-------------------------------------> Helper functions

def f(a : str) -> str:
    return a+"o"
def g(a : str) -> str:
    return a+"K"
def h(a : str) -> str:
    return a+a
def ag1(a : str, b : str) -> str:
    return a+"-"+b
def ag2(a : str, b : str) -> str:
    return b+"-"+a



In [6]:
flatten(["lkl","lk"])

'lkllk'

In [7]:
def test(ag,fl,fr,x,y):
    left = flatten(compose(x,fl)[1])
    right = flatten(compose(y,fr)[1])
    print(left+" - "+right)
    return ag(left,right)

In [35]:
fl = np.random.choice(transformation_functions(),5,replace=True)
fr = np.random.choice(transformation_functions(),5,replace=True)
ag = np.random.choice(similarity_functions(),1,replace=False)[0]
print(name(fl))
print(name(fr))
print(ag.__name__)
x = "The Boots and the Cats.  "
y = "Boots and Cats "
test(ag,fl,fr,x,y)


['remove_stopwords', 'tokenize', 'remove_whitespace', 'tokenize', 'strip_whitespace']
['remove_punctuation', 'flatten', 'flatten', 'tokenize', 'remove_whitespace']
damerau_levenshtein_dist_similarity
TheBootsCats. - BootsandCats


0.46153846153846156

In [12]:
def cos_similarity(x : str, y : str):
    # Compute cosine similarity using TF-IDF vectors
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([x, y])
    cosine_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]
    return cosine_sim



In [ ]:
#helper functions
def remove_elem(pop : list,score_pop : list,index_list : list[int]) -> None:
    for i in index_list:
        pop.remove(i)
        score_pop.remove(i)

In [ ]:
def elitism(self):
    #select best individuals
    score_index = list(np.argsort(self.population_scores))
    score_index.reverse()
    #elitism population
    best_individuals = deepcopy([self.population[index] for index in score_index[0:self.elitism_population_size]])
    #remove best elem from population
    self.population_new = np.append(self.population_new,best_individuals).tolist()
    #mutation population

def duel(self):
    index_candidate_pair = []
    while len(index_candidate_pair) != self.elitism_population_size:
        index_candidate_pair.append(np.random.choice(np.arange(0,self.population_size-1),2,replace=False))
    
    for i1,i2 in index_candidate_pair:
        if self.population_scores[i1] > self.population_scores[i2]:
            self.population_new.append(deepcopy(self.population[i1]))
        else:
            self.population_new.append(deepcopy(self.population[i2]))

In [ ]:
crossover_candidates = zip(
        (candidate_list := np.random.choice(self.population_candidate,(self.crossover_population_size*2)+1,replace=False))[0:self.crossover_population_size],
        candidate_list[self.crossover_population_size+1:])

In [ ]:
  self.elitism_population_size     : int = self.population_size - (
            self.mutation_population_size+
            self.crossover_population_size+
            self.random_tree_population_size
        )